# Property Defect Model Trainer
**Train YOLOv8 on 5-class wall defect dataset**

Classes: `crack`, `water_seepage`, `mold`, `peeling_paint`, `stairstep_crack`

1. Run all cells top to bottom
2. Download `best.pt` at the end
3. Put it in `weights/best.pt` on your machine

In [ ]:
# Step 0: Install dependencies
!pip install ultralytics roboflow -q

In [ ]:
# Step 1: Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU - go to Runtime > Change runtime type > T4 GPU'}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Step 2: Download dataset from Roboflow
import os
from roboflow import Roboflow

rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY", ""))
project = rf.workspace("objectdetection-qxiqx").project("detr_crack_dataset")
version = project.version(1)
dataset = version.download("yolov8")

print(f"\nDataset at: {dataset.location}")

In [ ]:
# Step 3: Train YOLOv8
from ultralytics import YOLO
import os

data_yaml = os.path.join(dataset.location, "data.yaml")

model = YOLO("yolov8n.pt")
results = model.train(
    data=data_yaml,
    epochs=50,
    batch=16,
    imgsz=640,
    project="runs",
    name="defect_model",
    exist_ok=True,
    patience=15,
    save=True,
    plots=True,
)

In [ ]:
# Step 4: Check results
best_path = "runs/defect_model/weights/best.pt"
model = YOLO(best_path)
print(f"Classes: {model.names}")
metrics = model.val(verbose=False)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# Step 5: Download best.pt
from google.colab import files
files.download("runs/defect_model/weights/best.pt")
print("\nDone! Put this file in weights/best.pt on your machine.")